In [66]:
from func_def import *
import keras

In [67]:
df = pd.read_csv('data_pre_processed.csv',index_col=[0]).reset_index(drop=True)
df.tail(3)

,url,event_url,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head,f_sig_str_body,f_sig_str_leg,f_sig_str_dist,f_sig_str_clinch,f_sig_str_ground,o_sig_str_head,o_sig_str_body,o_sig_str_leg,o_sig_str_dist,o_sig_str_clinch,o_sig_str_ground,date
7216,http://www.ufcstats.com/fight-details/e3a5a7ed...,http://www.ufcstats.com/event-details/ef61d9f5...,W,Nora Cornoll,Joselyne Edwards,Women's Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),12of37,17of24,11of16,17of39,23of38,0of0,6of20,8of9,4of13,14of35,4of6,0of1,30%,42%,27%,42%,57%,0%,33%,44%,22%,77%,22%,0%,2023-09-02 00:00:00
7217,http://www.ufcstats.com/fight-details/ac73c688...,http://www.ufcstats.com/event-details/ef61d9f5...,W,Farid Basharat,Kleydson Rodrigues,Bantamweight Bout,Submission,1,4:15,3 Rnd (5-5-5),2of3,2of2,0of1,0of1,1of1,3of4,3of4,2of2,2of2,4of5,1of1,2of2,50%,50%,0%,0%,25%,75%,42%,28%,28%,57%,14%,28%,2023-09-02 00:00:00
7218,http://www.ufcstats.com/fight-details/2a48d0a9...,http://www.ufcstats.com/event-details/ef61d9f5...,L,Zarah Fairn,Jacqueline Cavalcant,Catch Weight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),25of241,8of12,6of6,39of258,0of1,0of0,43of127,20of22,63of72,116of209,10of12,0of0,64%,20%,15%,100%,0%,0%,34%,15%,50%,92%,7%,0%,2023-09-02 00:00:00


In [68]:
# Pick features
FEATURES = ['head_sig_str','body_sig_str','leg_sig_str']

feature_columns = [f'f_{item}' for item in FEATURES] + [f'o_{item}' for item in FEATURES]
print(feature_columns)

['f_head_sig_str', 'f_body_sig_str', 'f_leg_sig_str', 'o_head_sig_str', 'o_body_sig_str', 'o_leg_sig_str']


In [69]:
data = df[['date','fighter','opponent','result'] + feature_columns].copy()

for prefix in ['f_','o_']:
    for feature in FEATURES:
        data[f'{prefix}{feature}'] = pd.to_numeric([int(item.split('of')[0]) for item in df[f'{prefix}{feature}']])
        data[f'{prefix}{feature}_att'] = pd.to_numeric([int(item.split('of')[1]) for item in df[f'{prefix}{feature}']])

data.tail(3)

,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att
7216,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,6,8,4,37,24,16,20,9,13
7217,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,3,2,2,3,2,1,4,2,2
7218,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,43,20,63,241,12,6,127,22,72


In [71]:
# Test
STATS = [item for item in FEATURES] + [f'{item}_att' for item in FEATURES]
for stat in STATS:
    data[f'f_{stat}_mean'] = [get_mean_stat(fighter=data.loc[index,'fighter'],stat_col=f'f_{stat}',time=data.loc[index,'date'],data=data) for index in range(len(data))]
    data[f'o_{stat}_mean'] = [get_mean_stat(fighter=data.loc[index,'opponent'],stat_col=f'o_{stat}',time=data.loc[index,'date'],data=data) for index in range(len(data))]

data.tail(3)

100%|██████████| 7219/7219 [00:00<00:00, 4794723.77it/s]


,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean
7216,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,6,8,4,37,24,16,20,9,13,0.000000,26.166667,0.0,27.0,0.000000,26.333333,0.0,71.333333,0.0,45.666667,0.000000,33.166667
7217,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,3,2,2,3,2,1,4,2,2,36.000000,23.500000,14.0,17.5,8.000000,6.500000,70.0,53.500000,17.0,21.500000,9.000000,7.500000
7218,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,43,20,63,241,12,6,127,22,72,34.333333,0.000000,6.0,0.0,3.666667,0.000000,84.0,0.000000,7.0,0.000000,3.666667,0.000000


In [ ]:
# same as above cell (normal loop not list comprehension)
STATS = [item for item in FEATURES] + [f'{item}_att' for item in FEATURES]
for stat in STATS:
    for index in tqdm(data.index):
        data.loc[index,f'f_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'fighter'],stat_col=f'f_{stat}',time=data.loc[index,'date'],data=data)
        data.loc[index,f'o_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'opponent'],stat_col=f'o_{stat}',time=data.loc[index,'date'],data=data)

data.tail(3)

  0%|          | 6/7219 [00:00<02:15, 53.30it/s]

100%|██████████| 7219/7219 [01:14<00:00, 97.47it/s] 


,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean
7216,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,6,8,4,37,24,16,20,9,13,0.000000,26.166667,0.0,27.0,0.000000,26.333333,0.0,71.333333,0.0,45.666667,0.000000,33.166667
7217,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,3,2,2,3,2,1,4,2,2,36.000000,23.500000,14.0,17.5,8.000000,6.500000,70.0,53.500000,17.0,21.500000,9.000000,7.500000
7218,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,43,20,63,241,12,6,127,22,72,34.333333,0.000000,6.0,0.0,3.666667,0.000000,84.0,0.000000,7.0,0.000000,3.666667,0.000000


In [72]:
for feature in FEATURES:
    data[f'{feature}_dif'] = data[f'f_{feature}'] - data[f'o_{feature}']
    data[f'{feature}_att_dif'] = data[f'f_{feature}_att'] - data[f'o_{feature}_att']
    data[f'{feature}_mean_dif'] = data[f'f_{feature}_mean'] - data[f'o_{feature}_mean']
    data[f'{feature}_att_mean_dif'] = data[f'f_{feature}_att_mean'] - data[f'o_{feature}_att_mean']

data.head(3)

,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean,head_sig_str_dif,head_sig_str_att_dif,head_sig_str_mean_dif,head_sig_str_att_mean_dif,body_sig_str_dif,body_sig_str_att_dif,body_sig_str_mean_dif,body_sig_str_att_mean_dif,leg_sig_str_dif,leg_sig_str_att_dif,leg_sig_str_mean_dif,leg_sig_str_att_mean_dif
0,2023-08-26,Max Holloway,Chan Sung Jung,W,34,31,10,25,6,3,77,35,16,110,8,6,78.407407,39.272727,26.777778,4.909091,10.444444,4.363636,194.592593,102.090909,35.703704,6.818182,13.629630,5.818182,9,-33,39.134680,92.501684,25,27,21.868687,28.885522,7,10,6.080808,7.811448
1,2023-08-26,Anthony Smith,Ryan Spann,W,44,12,35,39,14,6,92,15,40,158,23,6,21.052632,14.500000,6.210526,2.900000,4.789474,0.100000,51.473684,35.800000,9.842105,3.300000,5.368421,0.200000,5,-66,6.552632,15.673684,-2,-8,3.310526,6.542105,29,34,4.689474,5.168421
2,2023-08-26,Giga Chikadze,Alex Caceres,W,25,22,18,25,25,12,83,38,20,110,44,14,33.500000,31.500000,12.750000,11.714286,7.625000,5.892857,100.000000,72.714286,19.000000,15.607143,9.875000,7.214286,0,-27,2.000000,27.285714,-3,-6,1.035714,3.392857,6,6,1.732143,2.660714


In [54]:
X = [f'{feature}_mean_dif' for feature in FEATURES] + [f'{feature}_att_mean_dif' for feature in FEATURES]
y = 'result'
X

['head_sig_str_mean_dif',
 'body_sig_str_mean_dif',
 'leg_sig_str_mean_dif',
 'head_sig_str_att_mean_dif',
 'body_sig_str_att_mean_dif',
 'leg_sig_str_att_mean_dif']

In [55]:
set_train_test(X_columns=X, y_columns=y, data=data, test_size=0.3, valid_size=0)
data.tail(3)

Data split: Data=(7219, 41), train_set=(5012, 41), validation_set=(0, 41), test_set=(2207, 41)


,set,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean,head_sig_str_dif,head_sig_str_att_dif,head_sig_str_mean_dif,head_sig_str_att_mean_dif,body_sig_str_dif,body_sig_str_att_dif,body_sig_str_mean_dif,body_sig_str_att_mean_dif,leg_sig_str_dif,leg_sig_str_att_dif,leg_sig_str_mean_dif,leg_sig_str_att_mean_dif
7216,train,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,6,8,4,37,24,16,20,9,13,0.000000,26.166667,0.0,27.0,0.000000,26.333333,0.0,71.333333,0.0,45.666667,0.000000,33.166667,6,17,-26.166667,-71.333333,9,15,-27.0,-45.666667,7,3,-26.333333,-33.166667
7217,test,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,3,2,2,3,2,1,4,2,2,36.000000,23.500000,14.0,17.5,8.000000,6.500000,70.0,53.500000,17.0,21.500000,9.000000,7.500000,-1,-1,12.500000,16.500000,0,0,-3.5,-4.500000,-2,-1,1.500000,1.500000
7218,train,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,43,20,63,241,12,6,127,22,72,34.333333,0.000000,6.0,0.0,3.666667,0.000000,84.0,0.000000,7.0,0.000000,3.666667,0.000000,-18,114,34.333333,84.000000,-12,-10,6.0,7.000000,-57,-66,3.666667,3.666667


### Logistic regression model train

In [56]:
data.columns

Index(['set', 'date', 'fighter', 'opponent', 'result', 'f_head_sig_str',
       'f_body_sig_str', 'f_leg_sig_str', 'o_head_sig_str', 'o_body_sig_str',
       'o_leg_sig_str', 'f_head_sig_str_att', 'f_body_sig_str_att',
       'f_leg_sig_str_att', 'o_head_sig_str_att', 'o_body_sig_str_att',
       'o_leg_sig_str_att', 'f_head_sig_str_mean', 'o_head_sig_str_mean',
       'f_body_sig_str_mean', 'o_body_sig_str_mean', 'f_leg_sig_str_mean',
       'o_leg_sig_str_mean', 'f_head_sig_str_att_mean',
       'o_head_sig_str_att_mean', 'f_body_sig_str_att_mean',
       'o_body_sig_str_att_mean', 'f_leg_sig_str_att_mean',
       'o_leg_sig_str_att_mean', 'head_sig_str_dif', 'head_sig_str_att_dif',
       'head_sig_str_mean_dif', 'head_sig_str_att_mean_dif',
       'body_sig_str_dif', 'body_sig_str_att_dif', 'body_sig_str_mean_dif',
       'body_sig_str_att_mean_dif', 'leg_sig_str_dif', 'leg_sig_str_att_dif',
       'leg_sig_str_mean_dif', 'leg_sig_str_att_mean_dif'],
      dtype='object')

In [57]:
X

['head_sig_str_mean_dif',
 'body_sig_str_mean_dif',
 'leg_sig_str_mean_dif',
 'head_sig_str_att_mean_dif',
 'body_sig_str_att_mean_dif',
 'leg_sig_str_att_mean_dif']

In [58]:
# train model
X_train = data.loc[data['set'] == 'train', X].values
y_train = data.loc[data['set'] == 'train', y].values
    
# Create a logistic regression model
model = LogisticRegression(C=0.1, penalty='l2')

# Fit the model to the training data
model.fit(X_train, y_train)
model

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1)

In [59]:
# make predictions
data[f'{y}_pred'] = model.predict(data[X])
data.tail()

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,set,date,fighter,opponent,result,f_head_sig_str,f_body_sig_str,f_leg_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,f_head_sig_str_att,f_body_sig_str_att,f_leg_sig_str_att,o_head_sig_str_att,o_body_sig_str_att,o_leg_sig_str_att,f_head_sig_str_mean,o_head_sig_str_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_leg_sig_str_mean,o_leg_sig_str_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_leg_sig_str_att_mean,o_leg_sig_str_att_mean,head_sig_str_dif,head_sig_str_att_dif,head_sig_str_mean_dif,head_sig_str_att_mean_dif,body_sig_str_dif,body_sig_str_att_dif,body_sig_str_mean_dif,body_sig_str_att_mean_dif,leg_sig_str_dif,leg_sig_str_att_dif,leg_sig_str_mean_dif,leg_sig_str_att_mean_dif,result_pred
7214,test,2023-09-02 00:00:00,Taylor Lapilus,Caolan Loughran,W,59,18,7,28,13,7,133,23,7,100,23,9,32.500000,0.000000,11.75,0.0,4.250000,0.000000,87.5,0.000000,13.75,0.000000,5.500000,0.000000,31,33,32.500000,87.500000,5,0,11.75,13.750000,0,-2,4.250000,5.500000,W
7215,train,2023-09-02 00:00:00,Ange Loosa,Rhys McKee,W,71,19,8,48,19,10,144,29,9,145,25,13,61.000000,40.500000,20.50,18.0,27.000000,3.500000,164.5,89.500000,27.00,21.000000,31.500000,4.500000,23,-1,20.500000,75.000000,0,4,2.50,6.000000,-2,-4,23.500000,27.000000,W
7216,train,2023-09-02 00:00:00,Nora Cornoll,Joselyne Edwards,W,12,17,11,6,8,4,37,24,16,20,9,13,0.000000,26.166667,0.00,27.0,0.000000,26.333333,0.0,71.333333,0.00,45.666667,0.000000,33.166667,6,17,-26.166667,-71.333333,9,15,-27.00,-45.666667,7,3,-26.333333,-33.166667,W
7217,test,2023-09-02 00:00:00,Farid Basharat,Kleydson Rodrigues,W,2,2,0,3,2,2,3,2,1,4,2,2,36.000000,23.500000,14.00,17.5,8.000000,6.500000,70.0,53.500000,17.00,21.500000,9.000000,7.500000,-1,-1,12.500000,16.500000,0,0,-3.50,-4.500000,-2,-1,1.500000,1.500000,W
7218,train,2023-09-02 00:00:00,Zarah Fairn,Jacqueline Cavalcant,L,25,8,6,43,20,63,241,12,6,127,22,72,34.333333,0.000000,6.00,0.0,3.666667,0.000000,84.0,0.000000,7.00,0.000000,3.666667,0.000000,-18,114,34.333333,84.000000,-12,-10,6.00,7.000000,-57,-66,3.666667,3.666667,W


In [60]:
# Evaluate model
right_pred_test = len(data[(data.result_pred == data.result)&(data.set=='test')])
all_test = len(data[data.set=='test'])
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 0.6361576801087448
